In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/envs/det/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [22]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train2_split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512, 512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'train2_split_valid.json' # test json 정보
cfg.data.val.pipeline = cfg.data.train.pipeline

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512, 512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type="WandbLoggerHook",
         init_kwargs={"project":"level2_hiboostcamp",
                      "name" : "nahoon_valid_split_test",
                      "config" : {
                          "epochs" : 50,
                          "architecture" : "faster_rcnn",
                          "dataset" : "recycle"
                      }},
         interval=10,)
]
cfg.workflow = [('train', 1), ('val', 1)]


In [13]:
print(cfg.keys())

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'auto_scale_lr', 'seed', 'gpu_ids', 'work_dir', 'device'])


In [14]:
cfg.runner.max_epochs = 50

In [15]:
cfg.runner.max_epochs

50

In [16]:
# build_dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [17]:
cfg.log_config

{'interval': 50,
 'hooks': [{'type': 'TextLoggerHook'},
  {'type': 'WandbLoggerHook',
   'init_kwargs': {'project': 'level2_hiboostcamp',
    'name': 'nahoon_valid_split_test',
    'config': {'epochs': 50,
     'architecture': 'faster_rcnn',
     'dataset': 'recycle'}},
   'interval': 10,
   'log_checkpoint_metadata': True,
   'num_eval_images': 100}]}

In [18]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4010, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2867  | 1 [Paper]     | 3791  | 2 [Paper pack]  | 591   | 3 [Metal]   | 658   | 4 [Glass]    | 560   |
| 5 [Plastic]       | 1773  | 6 [Styrofoam] | 743   | 7 [Plastic bag] | 3531  | 8 [Battery] | 111   | 9 [Clothing] | 337   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [19]:
type(datasets[0])

mmdet.datasets.coco.CocoDataset

In [20]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-08 16:25:05,333 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-05-08 16:25:05,334 - mmcv - INFO - load model from: torchvision://resnet50
2023-05-08 16:25:05,335 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-05-08 16:25:05,431 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-05-08 16:25:05,447 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-08 16:25:05,475 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2023-05-08 16:25:05,481 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [23]:
# 모델 학습
train_detector(model, datasets, cfg, distributed=False, validate=False)

2023-05-08 16:25:25,925 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-08 16:25:26,365 - mmdet - INFO - Start running, host: root@7cf77f50633f, work_dir: /opt/ml/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2023-05-08 16:25:26,367 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                  

2023-05-08 16:26:05,233 - mmdet - INFO - Epoch [1][50/1003]	lr: 1.978e-03, eta: 1:47:52, time: 0.540, data_time: 0.057, memory: 7056, loss_rpn_cls: 0.4137, loss_rpn_bbox: 0.0446, loss_cls: 0.4459, acc: 91.6924, loss_bbox: 0.0701, loss: 0.9743, grad_norm: 6.3746
2023-05-08 16:26:28,317 - mmdet - INFO - Epoch [1][100/1003]	lr: 3.976e-03, eta: 1:39:38, time: 0.462, data_time: 0.010, memory: 7056, loss_rpn_cls: 0.1027, loss_rpn_bbox: 0.0469, loss_cls: 0.2620, acc: 95.3555, loss_bbox: 0.1762, loss: 0.5878, grad_norm: 2.4475
2023-05-08 16:26:51,616 - mmdet - INFO - Epoch [1][150/1003]	lr: 5.974e-03, eta: 1:36:55, time: 0.466, data_time: 0.011, memory: 7056, loss_rpn_cls: 0.1050, loss_rpn_bbox: 0.0601, loss_cls: 0.2951, acc: 94.3604, loss_bbox: 0.2122, loss: 0.6724, grad_norm: 2.1568
2023-05-08 16:27:14,800 - mmdet - INFO - Epoch [1][200/1003]	lr: 7.972e-03, eta: 1:35:14, time: 0.464, data_time: 0.011, memory: 7056, loss_rpn_cls: 0.0763, loss_rpn_bbox: 0.0414, loss_cls: 0.2841, acc: 94.2568, 

KeyboardInterrupt: 